# Cython functions

In [1]:
%load_ext Cython

## `def` functions in Cython: identical to `builtin_function_or_method`

* Expects Python objects as inputs; always returns a Python object.

In [12]:
def pyfunc(a, b):
    """Adds its arguments polymorphically, in pure Python."""
    return a + b

print(type(pyfunc))
print(dir(pyfunc))

<class 'function'>
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [13]:
%%cython

def cyfunc(a, b):
    """Adds its arguments polymorphically, in Cython!"""
    return a + b

print(type(cyfunc))
print(dir(cyfunc))

<class 'builtin_function_or_method'>
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']


### Cython functions don't have dynamic attributes that Python functions have

In [6]:
set(dir(pyfunc)) - set(dir(cyfunc))

{'__annotations__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__dict__',
 '__get__',
 '__globals__',
 '__kwdefaults__'}

In [16]:
assert pyfunc(1, 2) == cyfunc(1, 2)
assert pyfunc('a', 'b') == cyfunc('a', 'b')

class klass:
    def __add__(self, other):
        return 42
    
assert pyfunc(klass(), klass()) == cyfunc(klass(), klass())

## `cdef` functions: C-functions with Python-like syntax

In [30]:
%%cython

cdef untyped(a, b):
    return a + b

print(untyped(1, 2), untyped('a', 'b'))

cdef int typed(double a, double b):
    return <int>(a + b) # Type casting in Cython.

print(typed(1, 2), typed(3.14, 2.72))
# print(typed('a', 'b')) <<<<<<<<<<< Compilation Error, not ValueError / RuntimeError.

cdef struct st:
    int a, b

cdef st c_only_types(st *y, st *z):
    return st(y.a + z.a,
              y.b + z.b)


cdef st y = st(1, 2)
cdef st z = st(3, 4)
print(c_only_types(&y, &z))

3 ab
3 5
{'a': 4, 'b': 6}


### `cdef` variables / functions not visible to Python outside defining scope

In [31]:
untyped(1, 2), typed(1, 2), c_only_types

NameError: name 'untyped' is not defined

### Pop quiz / topics for discussion:

* Conjecture what code Cython creates when executing `print(typed(1, 2))` to make the `print()` call work.
* What does Cython do when we ask it to implicitly convert a `cdef struct` to a Python object?  Do you think it works in the other direction?

## `cpdef` functions: two functions in one!

In [38]:
%%cython

cdef struct st:
    int a, b
    

# cpdef functions are just like cdef functions with
# an implicitly defined Python wrapper for free.
# Note the cpdef and the pass-by-value here.
cpdef st c_only_types(st y, st z):
    return st(y.a + z.a,
              y.b + z.b)

# Call directly from other Cython code:
print(c_only_types(st(1, 2), st(3, 4)))

{'a': 4, 'b': 6}


In [39]:
# Call from Python (via Python wrapper)
c_only_types(dict(a=1, b=2), dict(a=3, b=4))

{'a': 4, 'b': 6}

### Pop quiz:

* given that a `cpdef` function is like a `cdef` function with the constraint of using only Python-convertible types in its signature, what C types can we not use with a `cpdef` function?
* Why are we able to use Python dictionaries as arguments with the `cpdef` `c_only_types()`?

## `cdef` and `cpdef` functions and error propogation

In [41]:
%%cython

cpdef int unchecked_div(int a, int b):
    return <int>(a / b)

# This result is concerning...
print(unchecked_div(1, 0))

0


Exception ignored in: '_cython_magic_cd7f0b0de30fcb6f379f89935edf0908.unchecked_div'
ZeroDivisionError: float division


In [43]:
%%cython

cpdef int checked_div(int a, int b) except *:
    return <int>(a / b)

# This is better...
print(checked_div(1, 0)) # ZeroDivisionError: float division

ZeroDivisionError: float division

### Raising exceptions inside cpdef / cdef functions with C return types

In [44]:
%%cython

cpdef int func() except -1: # we guarantee that -1 is never a valid return value, 
    # ...                   # so Cython can use it as a sentinel to flag that an 
    raise ValueError("...") # exception has occurred.

In [45]:
func()

ValueError: ...